# Graphical Pipelines

### Content of this Notebook:
* Understanding what are graphical pipelines
* Understanding the API of graphical pipelines
* Examples of simple pipelines and how they can be implemented with graphical pipelines.
* More complex graphical pipeline
    * Forecasting
* Grid search with a graphical pipeline



### What are Graphical Pipelines?
Recap sequential pipelines:

<img src="img/sequential_pipeline.png" width=750 />

Many tasks are non-sequential. To solve this two possibilities exist:
1. Nesting Sequential Pipelines.
2. Using Graphical Pipelines.


Thus, there is the generalised graphial pipeline.
* Graphical means that different steps may share the same predecessor or provide their outputs to the same successor (the dataflows can branch and merge).
<img src="img/graphical_pipeline.png" width=750 />


* Generalised means that the pipeline can be used for multiple tasks (e.g. forecasting, classification, ...).

**Note**

The graphical pipeline is still experimental. Thus, this graphical should not used in production. However, we would be happy to get feedback on the graphical pipeline.



### Potential Use-Cases
There exist various potential use-case for the graphical pipeline. In the following, we focus on a forecasting and a classification pipeline.
#### Forecasting Use-Case for Graphical Pipelines


The input of forecasters depends on the output of other forecasters, which same the same input.
* Forecaster should use the same preprocessing (branching of data flow)
* The forecasts should be combind and passed to the final forecaster (merging of data flow)

<img src="img/graphical_pipeline_example.png" width=750 />


#### Classification Use-Case for Graphical Pipelines


The input of classifier relies on different features and not all of these features are observable. Idea: Use a soft sensor that is a regressor for the non-observable features.

<img src="img/graphical_pipeline_softsensor.png" width=750 />

**Note:** The current experimental state of the graphical pipeline does not fully support this use-case. However, we are working on this. If you are interested in this use-case and want to contribute, please contact us.

### Credits
The graphical pipeline was first developed by pyWATTS [1] and was then adapted for sktime. The original implementation can be found [pyWATTS](https://github.com/KIT-IAI/pyWATTS). pyWATTS is a open source library developed at the Institute of Applied Informatics and Automation at the KIT and funded by HelmholtzAI.

> [1] Heidrich, Benedikt, et al. "pyWATTS: Python workflow automation tool for time series." arXiv preprint arXiv:2106.10157 (2021).

In [1]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from transformations.series.subset import ColumnSelect

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head, load_longley, load_macroeconomic
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    ColumnEnsembleForecaster,
    ForecastX,
    MultiplexForecaster,
    make_reduction,
)
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.sarimax import SARIMAX
from sktime.performance_metrics.forecasting import mean_absolute_error
from sktime.pipeline.pipeline import Pipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.exponent import ExponentTransformer

## How to build a Graphical Pipeline
Two ways to build a graphical pipeline:

1. Pass all steps to the pipeline during initialisation as for the sequential pipelines.

```python
pipeline = Pipeline([
    {"skobject": skobject1, "name": "bar", "edges": {"X": "y"}},
    {
        "skobject": skobject2,
        "name": "foo",
        "edges": {"X": "X", "y": "bar"},
    },
    {
        "skobject": skobject1,
        "name": "bar_inverse",
        "edges": {"X": "foo"},
        "method": "inverse_transform",
    },
])
```

2. Create a pipeline object and add the steps one by one.

```python

pipeline = Pipeline()
pipeline = pipeline.add_step(skobject1, "bar", edges={"X": "y"})
pipeline = pipeline.add_step(skobject2, "foo", edges={"X": "X", "y": "bar"})
pipeline = pipeline.add_step(
    skobject1, "bar_inverse", edges={"X": "foo"}, method="inverse_transform"
)
```

The `add_step`'s parameter or key of the dicts in the step list during initialisation are:

* skobject: The sktime object added to the pipeline
* name: The name of the step
* edges: The keys of the dictionary indicate the input of the skobject (X or y), and the values are the names of the steps that should be connected to the input argument. Note subsetting using `__` and feature union via lists are supported.
* method: The skobject's method that should be called. If not provided, the default method would be inferred based on the added skobject. This parameter is used for the inverse_transform method. Optional.
* kwargs: Additional keyword arguments passed to the sktime object. Optional.


## Examples
### Forecasting Pipeline
A simple forecasting pipeline implemented using the graphical pipeline.


In [2]:
general_pipeline = Pipeline()
differencer = Differencer()

general_pipeline = general_pipeline.add_step(
    differencer, "differencer", edges={"X": "y"}
)
general_pipeline = general_pipeline.add_step(
    SARIMAX(), "sarimax", edges={"X": "X", "y": "differencer"}
)
general_pipeline = general_pipeline.add_step(
    differencer, "differencer_inv", edges={"X": "sarimax"}, method="inverse_transform"
)

The pipeline can be visualised as follows:
<img src="img/forecasting_pipeline.png" width=750 />



In [3]:
y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

general_pipeline.fit(y=y_train, X=X_train, fh=[1, 2, 3, 4])
general_pipeline.predict(X=X_test)

1959    67213.735360
1960    68328.076304
1961    68737.861389
1962    71322.894013
Freq: A-DEC, Name: TOTEMP, dtype: float64

**Alternative Way in Defining the Pipeline**
An alternative to define a graphical pipeline would be to pass a list of steps to the Pipeline during creation. This would look as follows:

In [4]:
differencer = Differencer()

general_pipeline = Pipeline(
    [
        {"skobject": differencer, "name": "differencer", "edges": {"X": "y"}},
        {
            "skobject": SARIMAX(),
            "name": "sarimax",
            "edges": {"X": "X", "y": "differencer"},
        },
        {
            "skobject": differencer,
            "name": "differencer_inv",
            "edges": {"X": "sarimax"},
            "method": "inverse_transform",
        },
    ]
)

### Classification Pipeline
A simple classification pipeline implemented using the graphical pipeline.

In [5]:
general_pipeline = Pipeline()
general_pipeline = general_pipeline.add_step(
    ExponentTransformer(), "exponent", edges={"X": "X"}
)
general_pipeline = general_pipeline.add_step(
    KNeighborsTimeSeriesClassifier(), "classifier", edges={"X": "exponent", "y": "y"}
)

This pipeline can be visualised as follows:

<img src="img/classification_pipeline.png" width=750 />


In [6]:
X, y = load_arrow_head(split="train", return_X_y=True)
general_pipeline.fit(X=X, y=y)
general_pipeline.predict(X=X)

array(['0', '1', '2', '0', '1', '2', '0', '1', '2', '0', '1', '2', '0',
       '1', '2', '0', '1', '2', '0', '1', '2', '0', '1', '2', '0', '1',
       '2', '0', '1', '2', '0', '1', '2', '0', '1', '2'], dtype='<U1')

## A More Complex Example with Grid Search

The considered use-case is to forecast the inflation using forecasts of the real gross domestic product, real disposable personal income, and the unemployment rate. Furthermore the unemployment rate is forecasted using the same features except the unemployment rate itself.

<img src="img/graphical_pipeline_example.png" width=750 />


The data is taken from the macrodata dataset from the statsmodels package.


#### Pipeline Definition

In [7]:
pipeline = Pipeline()
sklearn_scaler = StandardScaler()
sktime_scaler = TabularToSeriesAdaptor(sklearn_scaler)
deseasonalizer = Deseasonalizer(sp=4)

pipeline = pipeline.add_step(
    sktime_scaler, name="scaler", edges={"X": "X__realgdp_realdpi_unemp"}
)
pipeline = pipeline.add_step(
    deseasonalizer, name="deseasonalizer", edges={"X": "X__realgdp_realdpi"}
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_gdp",
    edges={"y": "deseasonalizer__realgdp"},
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_dpi",
    edges={"y": "deseasonalizer__realdpi"},
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_unemp",
    edges={
        "y": "scaler__unemp",
        "X": [
            "forecaster_gdp",
            "forecaster_dpi",
        ],
    },
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_inflation",
    edges={"X": ["forecaster_dpi", "forecaster_unemp"], "y": "y"},
)

In [8]:
data = load_macroeconomic()

X = data[["realgdp", "realdpi", "unemp"]]
y = data[["infl"]]
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X=X, fh=fh)
X_train

,realgdp,realdpi,unemp
Period,,,
1959Q1,2710.349,1886.9,5.8
1959Q2,2778.801,1919.7,5.1
1959Q3,2775.488,1916.4,5.3
1959Q4,2785.204,1931.3,5.6
1960Q1,2847.699,1955.5,5.2
...,...,...,...
2005Q3,12683.153,9308.0,5.0
2005Q4,12748.699,9358.7,4.9
2006Q1,12915.938,9533.8,4.7


In [25]:
pipeline.fit(y=y_train, X=X_train, fh=fh)
result = pipeline.predict(X=None, fh=y_test.index)
result

C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.997e+02, tolerance: 1.843e-01
  model = cd_fast.enet_coordinate_descent(


,infl
Period,
2006Q4,2.188182
2007Q1,2.124281
2007Q2,1.045280
2007Q3,1.857716
2007Q4,1.790664
2008Q1,1.649457
2008Q2,1.874361
2008Q3,1.855627
2008Q4,1.858207


In [24]:
((result - y_test) ** 2).mean()

infl    18.648824
dtype: float64

#### Grid Search

This pipeline has multiple parameters that might be tested to find the configurations. These parameters include:
* which forecaster should be used for which variable -> `MultiplexForecaster`
* what should be the hyperparameters of the forecaster
* which features should be used for the different forecasters -> Tune the edges of the graphical pipeline!

<img src="img/graphical_pipeline_example_grid.png" width=750 />

Since we do forecasting, we use the ForecastingGridSearchCV.

1. Specify the parameter grid:

The keys of the dictionary are the parameters' in the pipeline, and the values specify which options should be tested.
Keys have the following structure: parameter of a step `<step_name>__skobject__<parameter-name>` and input edges of a step `<step-name>__edges_<Xory>`.


In [11]:
param_grid = {
    "forecaster_inflation__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_unemp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_dpi__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_gdp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_inflation__edges__X": [
        ["forecaster_unemp"],
        ["forecaster_unemp", "forecaster_dpi"],
    ],
    "forecaster_unemp__edges__X": [
        [],
        ["forecaster_dpi"],
        ["forecaster_gdp", "forecaster_dpi"],
    ],
    "deseasonalizer__edges__X": ["X__realgdp_realdpi", "scaler__realgdp_realdpi"],
}

Initialise the gridsearch using pipeline, cross-validation strategy, scoring, and param_grid.


In [12]:
gridcv = ForecastingGridSearchCV(
    pipeline,
    cv=SlidingWindowSplitter(
        window_length=len(X_train) - 20,
        step_length=4,
        fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ),
    scoring=mean_absolute_error,
    param_grid=param_grid,
)

Call fit on the gridsearch object.

In [13]:
gridcv.fit(y=y_train, X=X_train)

C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.644e+01, tolerance: 1.702e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.733e+01, tolerance: 1.701e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

ForecastingGridSearchCV(cv=SlidingWindowSplitter(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                     10, 11, 12],
                                                 step_length=4,
                                                 window_length=171),
                        error_score='raise',
                        forecaster=Pipeline(steps=[{'edges': {'X': 'X__realgdp_realdpi_unemp'},
                                                    'kwargs': {},
                                                    'method': None,
                                                    'name': 'scaler',
                                                    'skobject': TabularToSeriesAdaptor(transformer=StandardScaler())},
                                                   {'edges': {'X': 'X__realgdp_realdpi'},
                                                    'kwa...
                                    'forecaster_inflation__edges__X': [['forecaster_unemp'],
                                                                       ['forecaster_unemp',
                                                                        'forecaster_dpi']],
                                    'forecaster_inflation__skobject__selected_forecaster': ['ridge',
                                                                                            'lasso'],
                                    'forecaster_unemp__edges__X': [[],
                                                                   ['forecaster_dpi'],
                                                                   ['forecaster_gdp',
                                                                    'forecaster_dpi']],
                                    'forecaster_unemp__skobject__selected_forecaster': ['ridge',
                                                                                        'lasso']},
                        scoring=<function mean_absolute_error at 0x00000229820A81F0>)

In [14]:
gridcv.cv_results_

,mean_test__DynamicForecastingErrorMetric,mean_fit_time,mean_pred_time,params,rank_test__DynamicForecastingErrorMetric
0,1.539329,0.384369,0.050073,{'deseasonalizer__edges__X': 'X__realgdp_reald...,107.5
1,1.720565,0.127456,0.048370,{'deseasonalizer__edges__X': 'X__realgdp_reald...,119.5
2,1.394329,0.172980,0.079596,{'deseasonalizer__edges__X': 'X__realgdp_reald...,97.5
3,1.942051,0.102454,0.054941,{'deseasonalizer__edges__X': 'X__realgdp_reald...,129.5
4,2.033714,0.105911,0.055400,{'deseasonalizer__edges__X': 'X__realgdp_reald...,136.0
...,...,...,...,...,...
187,1.329079,0.084203,0.044186,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5
188,1.329079,0.084229,0.044309,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5
189,1.329079,0.078202,0.040972,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5
190,1.329079,0.086378,0.048652,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5


In [26]:
result = gridcv.predict(X=None, fh=y_test.index)
result

,infl
Period,
2006Q4,2.188182
2007Q1,2.124281
2007Q2,1.045280
2007Q3,1.857716
2007Q4,1.790664
2008Q1,1.649457
2008Q2,1.874361
2008Q3,1.855627
2008Q4,1.858207


In [27]:
((result - y_test) ** 2).mean()

infl    18.648824
dtype: float64

In [16]:
gridcv.best_params_

{'deseasonalizer__edges__X': 'X__realgdp_realdpi',
 'forecaster_dpi__skobject__selected_forecaster': 'ridge',
 'forecaster_gdp__skobject__selected_forecaster': 'ridge',
 'forecaster_inflation__edges__X': ['forecaster_unemp', 'forecaster_dpi'],
 'forecaster_inflation__skobject__selected_forecaster': 'lasso',
 'forecaster_unemp__edges__X': [],
 'forecaster_unemp__skobject__selected_forecaster': 'ridge'}

In [17]:
gridcv.best_score_

1.323560033949483

### How to implement a bit simpler version of the pipeline above by nesting sequential pipelines
* Simplifcation: The forecasting of the unemployment rate is not dependent on the GDP and DPI.
<img src="img/graphical_pipeline_simplified.png" width=750 />



Create sequential pipelines for forecasting the GDP, DPI and unemployment rate.

In [18]:
forecasting_pipeline_gdp = (
    ColumnSelect(["realgdp"])  # To train the forecaster only on the realgdp column
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)
forecasting_pipeline_dpi = (
    ColumnSelect(["realdpi"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

forecasting_pipeline_unemp = (
    ColumnSelect(["unemp"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

Use ColunmEnsembleForecaster to combine the forecasts of the DPI, GDP, UNEMP. (Union of forecasts)

In [19]:
input_inflation_forecast = ColumnEnsembleForecaster(
    [
        ("realdpi", forecasting_pipeline_dpi, "realdpi"),
        ("realgdp", forecasting_pipeline_gdp, "realgdp"),
        ("unemp", forecasting_pipeline_unemp, "unemp"),
    ]
)

Create the inflation forecaster.

In [20]:
inflation_forecast = ForecastX(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    input_inflation_forecast,
)

In [21]:
inflation_forecast.fit(y=y_train, X=X_train, fh=fh)

ForecastX(forecaster_X=ColumnEnsembleForecaster(forecasters=[('realdpi',
                                                              TransformedTargetForecaster(steps=[ColumnSelect(columns=['realdpi']),
                                                                                                 Deseasonalizer(),
                                                                                                 MultiplexForecaster(forecasters=[('ridge',
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                                        window_length=5)),
                                                                                                                                  ('lasso',
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                                                                                                        window_length...
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                                        window_length=5)),
                                                                                                                                  ('lasso',
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                                                                                                        window_length=5))])]),
                                                              'unemp')]),
          forecaster_y=MultiplexForecaster(forecasters=[('ridge',
                                                         RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                              window_length=5)),
                                                        ('lasso',
                                                         RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                              window_length=5))]))

In [22]:
inflation_forecast.predict()

,infl
2006Q4,3.979318
2007Q1,2.347512
2007Q2,1.443598
2007Q3,3.914533
2007Q4,2.533117
2008Q1,3.278010
2008Q2,3.861517
2008Q3,3.487510
2008Q4,4.195074
2009Q1,4.294984


In [23]:
inflation_forecast.get_params(True)

{'behaviour': 'update',
 'columns': None,
 'fh_X': None,
 'forecaster_X': ColumnEnsembleForecaster(forecasters=[('realdpi',
                                        TransformedTargetForecaster(steps=[ColumnSelect(columns=['realdpi']),
                                                                           Deseasonalizer(),
                                                                           MultiplexForecaster(forecasters=[('ridge',
                                                                                                             RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                  window_length=5)),
                                                                                                            ('lasso',
                                                                                                           

# Comparison graphical pipeline with nesting of sequential pipelines

### Advantages of graphical pipelines
* Enable an easy implementation of complex pipelines
    * By nesting sequential pipelines, even a simplified version of the graphical pipeline is very complicat to implement.
    * By nesting sequential pipelines, some graphical pipelines are not possible to implement (e.g., the example with coupled ForecastX).
* Preprocessing steps can not be shared between the different forecasters.
* The parameter structure can be very complex for the sequential pipelines.
* In a complex scenario, how would you fine-tune the edges?

### Advantages of sequential pipelines
* Constructing simple pipelines is very easy.
* Inverse operations are automatically applied.
* This is a mature feature compared to the experimental graphical pipeline.
